In [62]:
import pandas as pd
import re
from lxml import etree
import shutil
import os
import math
import copy
import glob
import numpy as np

In [63]:
#since i use back three
def read_xml_file(file_path_xml):
    result_xml_dict={}
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
#     pattern = r'\{.*?\}'
    for element in target_root.iter():
        if element.attrib:
            xpath_base = get_element_xpath(element)
            for e,val in element.attrib.items():
                xpath = xpath_base + ' / ' + '@' + e
#                 print(f'-->{xpath}')
                result_xml_dict[xpath] = tree.getpath(element)
        else:
            xpath = get_element_xpath(element)
            result_xml_dict[xpath] = tree.getpath(element)
    return result_xml_dict
        
def get_element_full_path(tree,element):
    return tree.getpath(element)

def read_xml_file_2(file_path_xml):
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
    for element in target_root.iter():
        xpath = tree.getpath(element)
        print(f"XPath: {xpath}")
def load_base_value_iden(file_path):
    if os.path.exists(file_path):
        
        # File exists, load the variable from the file
        if os.path.getsize(file_path) > 0:
                    # File is not empty, load the variable from the file
            with open(file_path, 'r') as file:
                
                variable_loaded = file.read().strip()
#                 print('loading')
#                 print(variable_loaded)
        else:
            # File exists but is empty, initialize with a default value
            variable_loaded = "11e1d6e4-74ba-49cc-9b55-1"
    else:
        # File does not exist, create the file and write the variable to it
#         print('+++++')
        print('creating file...')
        variable_to_store = "11e1d6e4-74ba-49cc-9b55-1"
        with open(file_path, 'w') as file:
            file.write(variable_to_store)
        variable_loaded = variable_to_store
        
    return variable_loaded
def save_base_value_iden(file_path,variable_to_store):
    
    # Write the variable to the file
    with open(file_path, 'w') as file:
        file.write(variable_to_store)
def increment_value_iden(value,file_path):

    str_last = value.split('-')[-1]
    last= int(str_last)
    last+=1
    
    exclude_last = value.split('-')[:-1]
    temp = '-'.join(exclude_last)

    new_identifier = temp + '-' + str(last)
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            variable_to_store = new_identifier
#             print('writing')
#             print(variable_to_store)
            file.write(variable_to_store)
    else:
        print('load base value first')
def adjust_identifier(root_copy,filepath_base_identifier):
    base_identifier = load_base_value_iden(filepath_base_identifier)

    for index,e in enumerate(root_copy.iter()):
        if index == 10:#rsender identifier:
            e.text = base_identifier
            increment_value_iden(base_identifier,filepath_base_identifier)
def extract_out_second_half(actual_xpath):#to be attachrd to path_base
    temp = actual_xpath.split('/')[-1]
    return temp
def handle_multiple(current_test_case,test_case_id,r_h,target_root):
    print(f' {test_case_id}  ')
    
    hold_1 = prepare_data_for_multiple(current_test_case)
    print('------------')
    print(hold_1)
    print('   ')
    field_1 = hold_1[0][0]
    base_path_field_1 = handle_base_path(field_1)
    
    
    print('------------')
    print('   ')
    temp_base_path = base_path_field_1
    for index,each_case in enumerate(hold_1):
        path_base = temp_base_path + f'[{index+1}]' + '/'
        for each_path in each_case:
            base_path_each_path = handle_base_path(each_path)
            if base_path_each_path == base_path_field_1:
                
                print('   ')
                print(f'----<<--Ammend base path for {each_path} ->>-----')
                if each_path in r_h:
                    actual_xpath = r_h[each_path]
                    temp = extract_out_second_half(actual_xpath)
                    path_base_final = path_base + temp
                    print(f'----<<--actual amended path ==>{path_base_final}')
                    target_element = target_root.xpath(path_base_final,namespaces = namespace_map)
                    print(f'----<<--actual element==>{target_element}')
                    print('   ')
                    print(f'----<<--actual path {each_path}')
                    if '@' in each_path:
                        print('   ')
                        print('------change attrib------')
                        if len(target_element)>0:
                            print(f'----<<--actual element value==>{target_element[0].attrib}')
                            print('   ')
                        else:
                             print(f'--cannot get--<<--{each_path} | {target_element} is empty ')
                    else:
                        print('   ')
                        print('------change element------')
                        if len(target_element)>0:
                            print(f'----<<--actual element text==>{target_element[0].text}')
                        else:
                             print(f'---cannot get-<<--{each_path} | {target_element} is empty ')
                    print('   ')
            else:#base_path_each_path != base_path_field_1:
                print('   ')
                print(f'----<<--just change straight away {each_path} ->>')
                print('   ')
                if '@' in each_path:
                    print('   ')
                    print('------change attrib------')
                    print('   ')
                else:
                    print('   ')
                    print('------change element------')
                    print('   ')
                
                
    print('   ')
    print('------------')
    print('   ')
def handle_base_path(s):
    start = r_h[s]
#     print(f'start {start}')
    a= start
    specific_elem = a.split('/')[-1]
#     print(f'specific elem{specific_elem}')
    temp = a.split('/')[:-1]
#     print('temp',temp)
    temp_a = '/'.join(temp)
#     print('temp_a',temp_a)

    temp_b = temp[-1].split('[')[0]
    temp_c = temp
    temp_c[-1] = temp_b
    temp_d = '/'.join(temp_c)
#     print('temp_d',temp_d)
#     print('   ')
    return temp_d
            
def evaluate_if_got_multiple(current_test_case):
    occur_again = False
    for index,fields in enumerate(current_test_case):
        xpath_find = fields[0].strip()
        if index == 0:
            field_one = xpath_find
        if (xpath_find == field_one) and index != 0:
            occur_again = True
            return True
    return occur_again
def prepare_data_for_multiple(current_test_case):
    holding = []
    for index,fields in enumerate(current_test_case):
        
        path_to_store = fields[0]

        if index == 0:
            temp = []
            field_one = path_to_store
            temp.append(path_to_store)
            
        elif (path_to_store == field_one) and (index != 0):
            holding.append(temp)
            temp = []

        
            temp.append(path_to_store)
        else:
#             print('  ----  ')
#             print('    ')
#             print(temp)
#             print('  ----  ')
#             print('    ')
            temp.append(path_to_store)
    holding.append(temp)
    return holding
def fix_space_typo(temp):
    temp_2 = temp.split("/")
    for index,i in enumerate(temp_2):
        temp_2[index] = i.strip()
    temp_3 = ' / '.join(temp_2)
    return temp_3

In [64]:
sheet_name = '6_Schema Validation'
folder_path = '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_' + sheet_name
#error prone files from 3_code validation
problems=[i for i in range(1,113)]


In [65]:
folder_path

'/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation'

In [66]:
def get_id(xml_file):
    temp = xml_file.split('/')[-1]
    temp_2 = temp.split('_Test_case_')[-1].split('_')[0]
    return temp_2
def process_xml_files_3(folder_path):
    c=0
    map_to_values ={}#so can overwrite the dirty files in the oirinal 3_condtionial
    check_how_many_removed = []
    xml_files = glob.glob(os.path.join(folder_path, '*.xml'))
    for file in xml_files:
        print(file)
        test_id = get_id(file)
        if int(test_id) in problems:
            c+=1
            map_to_values[test_id] = file
    return map_to_values

In [67]:
mtv = process_xml_files_3(folder_path)

/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_76_6_Schema Validation.xml
/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_79_6_Schema Validation.xml
/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_45_6_Schema Validation.xml
/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_94_6_Schema Validation.xml
/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_34_6_Schema Validation.xml
/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_19_6_Schema Validation.xml
/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_87_6_Schema Validation.xml
/Users/cheongray/iras_api_clean_1/6_Schema_Valid

In [68]:
mtv

{'76': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_76_6_Schema Validation.xml',
 '79': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_79_6_Schema Validation.xml',
 '45': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_45_6_Schema Validation.xml',
 '94': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_94_6_Schema Validation.xml',
 '34': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_34_6_Schema Validation.xml',
 '19': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_19_6_Schema Validation.xml',
 '87': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_87_6_Sc

In [69]:
def check_xpath_exists(invoice_dict,xpath):
    if xpath in invoice_dict:
        return True
    else:
        return False
def insert_invoice_dict(invoice_dict_xpath,insert_value,xpath_exists):
    if not xpath_exists:
        invoice_dict[xpath] = insert_value
    else:
        invoice_dict[xpath].append(inse)
    

def get_mapping_for_xml_invoice(file_path_of_sample_xml):

    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:Invoice-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2"
    #     'xmlns' : "urn:oasis:names:specification:ubl:schema:xsd:Invoice-2"
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
    # format_key_map = []
    invoice_dict={}
    for element in target_root.iter():
        if isinstance(element, etree._Comment):
            continue
        ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

        string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

        path = ' / '.join(string_ancestors)
        for prefix, uri in namespace_map.items():
            path = path.replace('{' + uri + '}', prefix + ':')
        if 'sb:StandardBusinessDocument / ubl:Invoice / ' in path:
            path = path.replace('sb:StandardBusinessDocument / ', '')
            if element.attrib:
                xpath_base = path
#                 print(f'==={1}======')
#                 print(element)
#                 print(element.text)
#                 print(xpath_base)
#                 print('============')
#                 print('           ')
                xpath_exists = check_xpath_exists(invoice_dict,xpath_base)
                if not xpath_exists:
                    invoice_dict[xpath_base] = [tree.getpath(element)]
                    
                else:
                    invoice_dict[xpath_base].append(tree.getpath(element))
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
                    if xpath in invoice_dict:
                         invoice_dict[xpath].append(tree.getpath(element))
                    else:
                        invoice_dict[xpath] = [tree.getpath(element)]
            else:                
#                 print(f'={909}=={1}======')
#                 print(element)
                
#                 print(element.text)
#                 print(xpath_base)
#                 print(path)
#                 print('============')
#                 print('           ')
                insert_value = tree.getpath(element)
                xpath_exists = check_xpath_exists(invoice_dict,path)
                if not xpath_exists:
                    invoice_dict[path] = [insert_value]
                else:
                    invoice_dict[path].append(insert_value)

        else:
            if element.attrib:
#                 print(f'==={2}======')
#                 print(element)
#                 print(path)
#                 print('============')
#                 print('           ')
                xpath_base = get_element_xpath(element)
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
    #                 print(f'-->{xpath}')


                    invoice_dict[xpath] = [tree.getpath(element)]
            else:
#                 print(f'==={2}======')
#                 print(element)
#                 print(path)
#                 print('============')
#                 print('           ')
                xpath = get_element_xpath(element)#defined on top
                

                
                insert_value = tree.getpath(element)
#                 print('=========')
#                 print(element)
#                 print(xpath)
#                 print(f'insert val = {insert_value}')
#                 print('============')
#                 print('           ')
                xpath_exists = check_xpath_exists(invoice_dict,xpath)
                if not xpath_exists:
                    invoice_dict[xpath] = [insert_value]
                else:
                    
                    invoice_dict[xpath].append(insert_value)
    r_h = invoice_dict
    return r_h
def get_mapping_for_xml_credit(file_path_of_sample_xml):

    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:CreditNote-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2" 
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
    # format_key_map = []
    invoice_dict={}
    for element in target_root.iter():
        if isinstance(element, etree._Comment):
            continue
        ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

        string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

        path = ' / '.join(string_ancestors)
        for prefix, uri in namespace_map.items():
            path = path.replace('{' + uri + '}', prefix + ':')
        if 'sb:StandardBusinessDocument / ubl:CreditNote / ' in path:

            path = path.replace('sb:StandardBusinessDocument / ', '')
            if element.attrib:
                xpath_base = path
#                 print(f'==={1}======')
#                 print(element)
#                 print(element.text)
#                 print(xpath_base)
#                 print('============')
#                 print('           ')
                xpath_exists = check_xpath_exists(invoice_dict,xpath_base)
                if not xpath_exists:
                    invoice_dict[xpath_base] = [tree.getpath(element)]
                    
                else:
                    invoice_dict[xpath_base].append(tree.getpath(element))
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
                    if xpath in invoice_dict:
                         invoice_dict[xpath].append(tree.getpath(element))
                    else:
                        invoice_dict[xpath] = [tree.getpath(element)]
            else:                
#                 print(f'={909}=={1}======')
#                 print(element)
                
#                 print(element.text)
#                 print(xpath_base)
#                 print(path)
#                 print('============')
#                 print('           ')
                insert_value = tree.getpath(element)
                xpath_exists = check_xpath_exists(invoice_dict,path)
                if not xpath_exists:
                    invoice_dict[path] = [insert_value]
                else:
                    invoice_dict[path].append(insert_value)

        else:
            if element.attrib:
#                 print(f'==={2}======')
#                 print(element)
#                 print(path)
#                 print('============')
#                 print('           ')
                xpath_base = get_element_xpath(element)
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
    #                 print(f'-->{xpath}')


                    invoice_dict[xpath] = [tree.getpath(element)]
            else:
#                 print(f'==={2}======')
#                 print(element)
#                 print(path)
#                 print('============')
#                 print('           ')
                xpath = get_element_xpath(element)#defined on top
                

                
                insert_value = tree.getpath(element)
#                 print('=========')
#                 print(element)
#                 print(xpath)
#                 print(f'insert val = {insert_value}')
#                 print('============')
#                 print('           ')
                xpath_exists = check_xpath_exists(invoice_dict,xpath)
                if not xpath_exists:
                    invoice_dict[xpath] = [insert_value]
                else:
                    
                    invoice_dict[xpath].append(insert_value)
    r_h = invoice_dict
    return r_h






def credit_or_invoice(file_path):
#     print('********')
#     print(file_path)
    tree = etree.parse(file_path)
    target_root = tree.getroot()
    for index,e in enumerate(target_root.iter()):
        if index ==11:
            if 'Credit' in e.text:
                return 'credit'
            elif 'Invoice' in e.text:
                return 'invoice'
            else:
                return 'undefined'
def proper_mapping_final(file_path):

    if credit_or_invoice(file_path) == 'credit':
        r_h = get_mapping_for_xml_credit(file_path)
        return r_h
    elif credit_or_invoice(file_path) == 'invoice':
        r_h = get_mapping_for_xml_invoice(file_path)
        return r_h
    else:
        return None
def get_element_xpath(element):
    path = [element.tag.split('}')[-1]]
    for parent in element.iterancestors():
        parent_tag = parent.tag.split('}')[-1]  
        path.insert(0, parent_tag)
    return ' / '.join(path)

In [70]:
file_path = r"/Users/cheongray/iras_api_pytest_1/AP_SP_files/AP_SP_FINAL_5.xlsx"#download to here from emails

# path_to_sample_xml_files_folder = r"/Users/cheongray/iras_api_clean_1/sample_xml_files"


df_s = pd.read_excel(file_path, sheet_name=sheet_name)
df_s_temp = df_s.copy()
df_s_temp['Test Case ID'] = df_s_temp.index.astype(float)#REMINDER,now test_case_id is 1_1
# df_s_peppol = df_s_temp[(df_s_temp['Standard'] == 'PEPPOL') & (df_s_temp['Document Type'] == 'Invoice')].copy()
# df_s_peppol['Test Case ID'] = df_s_peppol['Test Case ID'] + 1
# df_s_peppol.reset_index(drop=True)
df_s_temp['Test Case ID'] = df_s_temp['Test Case ID'] + 1

In [71]:

df_s_temp_subset = df_s_temp.copy()

unique_descriptions = set()

target_elements_syn = []

for index,row in df_s_temp_subset.iterrows():
    filled_columns = row.dropna().index
    test_case_id = f' {index}'
    descript = row['Test Case Description']
#     print(f'Test case {index+1}')
#     print(f'Test case description : {descript}')
    unique_descriptions.add(descript)#just to build function_mapping
    temp={}
    temp['Rule ID'] = row['Rule ID']
    temp['Standard'] = row['Standard']
    temp['Document Type'] = row['Document Type']
    temp['BIS/PINT'] = row['BIS/PINT']
    temp['Test Case ID'] = int(row['Test Case ID'])
    id = int(row['Test Case ID'])
    temp['descript'] = descript
    temp['Expected Result'] = row['Expected Result Response (for 400 response)']
    temp['Status'] = row['Expected Result']
    temp['Fields'] = []
    temp['Sample'] = row['Sample']
#     temp['Helper'] = row['Helper']
    for i, col in enumerate(filled_columns, start=1):
        field_name = f'Field {i}'
        
        if field_name in df_s_temp_subset.columns:
            # Process the data as needed
#             print(f'Column: {field_name}, Value: {row[field_name]}')
   
            
            temp_2 = [] #each field will be one array,can do dict but mor steps when want access
            if not pd.isna(row[field_name]):
                field_value = field_name + ' Value'
                field_name_insert = fix_space_typo(row[field_name])
                temp_2.append(field_name_insert)
                temp_2.append(row[field_value])
                temp['Fields'].append(temp_2)
        
    target_elements_syn.append(temp)

            


In [72]:
path_to_sample_xml_files_folder = '/Users/cheongray/iras_api_pytest_1/sample_xml_files'
#gets the needed files for 2_Conditional_Input_Validation
file_paths_xml_files = {}

for i,test_case in enumerate(target_elements_syn):
    
    sample_to_use = test_case['Sample']
    
    num = sample_to_use.split(' ')[-1]
    file_name = 'sample' + '_' + num + '.xml'
    
    path_to_sample_xml_files = path_to_sample_xml_files_folder + '/' + file_name
    
    file_paths_xml_files[sample_to_use] = path_to_sample_xml_files


In [73]:
# tree = etree.parse('/Users/cheongray/iras_api_pytest_1/sample_xml_files/sample_2.xml')
# target_root = tree.getroot()
# for t in target_root.iter():
#     print(t)
#     print(get_element_xpath(t))
#     print('=============')
#     print('         ')

In [74]:
work_st_1 = []
for testcase in target_elements_syn:
    if testcase['Test Case ID'] in problems:
        temp=[]
        temp.append(testcase['Test Case ID'])
        temp.append(testcase['Fields'])
        temp.append(testcase['Sample'])
        temp.append(testcase['descript'])
        work_st_1.append(temp)

In [75]:
work_st_2 = []
for index_work_1,e in enumerate(work_st_1):
    if int(work_st_1[index_work_1][0]) in problems:
        work_st_2.append(work_st_1[index_work_1])

In [76]:
mtv

{'76': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_76_6_Schema Validation.xml',
 '79': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_79_6_Schema Validation.xml',
 '45': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_45_6_Schema Validation.xml',
 '94': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_94_6_Schema Validation.xml',
 '34': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_34_6_Schema Validation.xml',
 '19': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_19_6_Schema Validation.xml',
 '87': '/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_87_6_Sc

In [77]:
for test_case in work_st_2:
    t = test_case[0]
    print(t)
#     f_p = mtv[str(t)]
#     print(f_p)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112


In [78]:
# def flip_bit(value):#just in caes so never go out of lenght 2 range
#     return value ^ 1

In [79]:
work_st_2

[[1,
  [['ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentTypeCode',
    'sgdtotal-excl-gst'],
   ['ubl:Invoice / cac:AdditionalDocumentReference / cbc:ID', 'SGD'],
   ['ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentDescription',
    299],
   ['ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentTypeCode',
    'sgdtotal-incl-gst'],
   ['ubl:Invoice / cac:AdditionalDocumentReference / cbc:ID', 'SGD'],
   ['ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentDescription',
    350.0]],
  'Sample 2',
  "Exactly 1 instance of 'sgdtotal-excl-gst' exists when document currency code != 'SGD' with ID = 'SGD' and DocumentDescription provided as amount"],
 [2,
  [['ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentTypeCode',
    'sgdtotal-excl-gst'],
   ['ubl:Invoice / cac:AdditionalDocumentReference / cbc:ID', 'SGD'],
   ['ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentDescription',
    299],
   ['ubl:Invoice / cac:AdditionalDocume

In [80]:
def flip_bit(value):#just in caes so never go out of lenght 2 range
    return value ^ 1
namespace_map = {
    'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
    'xs': 'http://www.w3.org/2001/XMLSchema',
    'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
    }
for test_case in work_st_2:
    
    desc = test_case[3]
    print('============Description===============')
    print(desc)
    print('           ')
    print('            ')
    sample_number = test_case[2]
    test_case_id = test_case[0]
    f_p = mtv[str(test_case_id)]
    print(f'===={sample_number}====={f_p}={test_case_id}==')
    a = proper_mapping_final(f_p)
    holding_set = []
    values = []
    pos_temp = 0#used to access the first/second value
    pos_temp_storage={}#each field will have a flip bit
    key_storage_for_pos_temp_storage = []
    for i,path in enumerate(test_case[1]):
    
        pos_temp_storage[path[0]] = pos_temp
        key_storage_for_pos_temp_storage.append(path[0]+str(i))

    print(pos_temp_storage)
    for i,path in enumerate(test_case[1]):
        unique_iden = str(i)
        xpaths = a[path[0]]
        value = path[1]
        if i == 0 and str(value) == 'nan':
            if 'missing value' or 'No instance' in desc:
                print(f'chaniging value for {test_case_id} ')
                value = ''
        values.append(value)
#         print( ' xpaths')
#         print(xpaths)
#         print('========')
        if isinstance(xpaths, str):
            if xpaths not in holding_set:
                holding_set.append(xpaths)
        else:
            key = pos_temp_storage[path[0]]
#             print(xpaths[pos_temp_storage[path[0]+unique_iden]])
            print(key)
            print(xpaths)
            if len(xpaths) <2:
                holding_set.append(xpaths[0])
            else:
                holding_set.append(xpaths[key])
            pos_temp_storage[path[0]] = flip_bit(key)
#         print('&&&&&&&')
#         print(pos_temp_storage)
#         print('&&&&&&&&')
            
            
#     print('         ')
        
#     print('    holding set      ')
#     print(holding_set)
    tree = etree.parse(f_p)
    target_root = tree.getroot()
    print('   ')
    original_tree_copy = copy.deepcopy(target_root)
    for index_values,path_to_insert in enumerate(holding_set):
        attrib_yes = False
        key_for_pos_temp_storage = key_storage_for_pos_temp_storage[index_values]
        
        element =  original_tree_copy.xpath(path_to_insert,namespaces = namespace_map)[0]
        insert_value = str(values[index_values])
        if '@' in key_for_pos_temp_storage :#attribute
                attrib_yes = True
                for key, value in element.attrib.items():
                    print(f'----<<--actual element value==>{element.attrib}')
                    element.attrib[key] = insert_value
                    print(f'----<<--new element value==>{element.attrib}')
        
        if '@' not in key_for_pos_temp_storage:
            print(f'original text {element.text} index values={index_values} {insert_value}')
            element.text =insert_value
            print(f'new text {element.text}')
            print('   ')
            
        if index_values == 0:
            if insert_value == 'nan' or insert_value == '':
                if 'does not exist' in desc:
                    #remove attribute/element
                    print('does not exist in description....')
                    if not attrib_yes:
                        print('%%%%%%%% Remove element %%%%%%%%')
                        parent = element.getparent()

                        if element.getparent() is not None:
                            element.getparent().remove(element)

                        print('------------')
                    else:
                        print('%%%%%%%% Remove attrib %%%%%%%%')
                        del element.attrib[key_attrib]
                
            
    tree = etree.ElementTree(original_tree_copy)
    filepath = mtv[str(test_case_id)]
    tree.write(filepath, encoding='utf-8', xml_declaration=True)
    
    print('============')

============Description===============
Exactly 1 instance of 'sgdtotal-excl-gst' exists when document currency code != 'SGD' with ID = 'SGD' and DocumentDescription provided as amount
           
            
====Sample 2=====/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_1_6_Schema Validation.xml=1==
{'ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentTypeCode': 0, 'ubl:Invoice / cac:AdditionalDocumentReference / cbc:ID': 0, 'ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentDescription': 0}
0
['/*/*[2]/cac:AdditionalDocumentReference[1]/cbc:DocumentTypeCode', '/*/*[2]/cac:AdditionalDocumentReference[2]/cbc:DocumentTypeCode', '/*/*[2]/cac:AdditionalDocumentReference[4]/cbc:DocumentTypeCode']
0
['/*/*[2]/cac:AdditionalDocumentReference[1]/cbc:ID', '/*/*[2]/cac:AdditionalDocumentReference[2]/cbc:ID', '/*/*[2]/cac:AdditionalDocumentReference[3]/cbc:ID', '/*/*[2]/cac:AdditionalDocumentReference[4]/cbc:ID']
0

   
original text sgdtotal-excl-gst index values=0 sgdtotal-excl-gst
new text sgdtotal-excl-gst
   
original text SGD index values=1 SGD
new text SGD
   
original text 230.0 index values=2 Two Hundred and Ninety Nine
new text Two Hundred and Ninety Nine
   
original text sgdtotal-incl-gst index values=3 sgdtotal-incl-gst
new text sgdtotal-incl-gst
   
original text SGD index values=4 SGD
new text SGD
   
original text 245.0 index values=5 350.0
new text 350.0
   
============Description===============
Exactly 1 instance of 'sgdtotal-excl-gst' exists when document currency code != 'SGD' with ID = 'SGD' and DocumentDescription missing value
           
            
====Sample 4=====/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-015_Test_case_41_6_Schema Validation.xml=41==
{'ubl:CreditNote / cac:AdditionalDocumentReference / cbc:DocumentDescription': 0, 'ubl:CreditNote / cac:AdditionalDocumentReference / cbc:DocumentTypeCode': 0, 'ubl:Credi

   
original text 230.0 index values=0 
new text 
   
does not exist in description....
%%%%%%%% Remove element %%%%%%%%
------------
original text sgdtotal-excl-gst index values=1 sgdtotal-incl-gst
new text sgdtotal-incl-gst
   
original text SGD index values=2 SGD
new text SGD
   
original text sgdtotal-incl-gst index values=3 sgdtotal-excl-gst
new text sgdtotal-excl-gst
   
original text SGD index values=4 SGD
new text SGD
   
original text 245.0 index values=5 200.0
new text 200.0
   
============Description===============
Exactly 1 instance of 'sgdtotal-incl-gst' exists when document currency code != 'SGD' with ID = 'SGD' and DocumentDescription provided as amount
           
            
====Sample 4=====/Users/cheongray/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/IRASC5-016_Test_case_85_6_Schema Validation.xml=85==
{'ubl:CreditNote / cac:AdditionalDocumentReference / cbc:DocumentTypeCode': 0, 'ubl:CreditNote / cac:AdditionalDocumentReference / cbc:ID':

In [81]:
# holding_paths = a['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID']

In [61]:
#---end

In [151]:
# path_to_sample_xml_files = '/Users/cheongray/iras_api_pytest_1/sample_xml_files'
#gets the needed files for 2_Conditional_Input_Validation
file_paths_xml_files = {}

for i,test_case in enumerate(target_elements_syn):
    
    sample_to_use = test_case['Sample']
    
    num = sample_to_use.split(' ')[-1]
    file_name = 'sample' + '_' + num + '.xml'
    
    path_to_sample_xml_files = path_to_sample_xml_files_folder + '/' + file_name
    
    file_paths_xml_files[sample_to_use] = path_to_sample_xml_files


In [223]:
file_paths_xml_files

{'Sample 1': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml',
 'Sample 3': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml',
 'Sample 11': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_11.xml',
 'Sample 9': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_9.xml',
 'Sample 10': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_10.xml'}

In [188]:
final

{2858: ['*',
  'urn:peppol:bis:Payables',
  ['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID',
   0],
  ['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID',
   1],
  ['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID',
   1],
  ['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID',
   0],
  ['ubl:Invoice / cbc:ProfileID', 0],
  ['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID',
   0],
  ['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID',
   1],
  ['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID',
   1],
  ['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID',
   0],
  ['ubl:Invoice / cbc:ProfileID', 0],
  ['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID',
   0],
  ['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID',
   1],
  ['ubl:

In [109]:
final

[['/*/*[2]/cac:InvoiceLine[1]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID',
  'SRCA-S'],
 ['/*/*[2]/cac:InvoiceLine[1]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID',
  'ZR']]

In [66]:
work_st_1[0][-1]

[['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID',
  'SRCA-S'],
 ['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID',
  'ZR'],
 ['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID',
  'SRCA-S'],
 ['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID',
  'ZR'],
 ['ubl:Invoice / cbc:ProfileID',
  'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0 ']]

In [111]:
for paths in work_st_1[index_work_1][1]:
    print(a[paths[0]])

['/*/*[2]/cac:InvoiceLine[1]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID', '/*/*[2]/cac:InvoiceLine[2]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID']
['/*/*[2]/cac:InvoiceLine[1]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID', '/*/*[2]/cac:InvoiceLine[2]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID']


In [ ]:
# #     if paths in worked_on:#means similar,then choose the next path in 'a' dict
# #         pass
# #     else:
# #         worked_on.append(paths)
# #         pos_for_path = 0 # first one take first value
# #         xpath = holding_paths[pos_for_path] 
# #         temp = []
# #         temp.append(xpath,)
        
        
        
# #     for incre,different_similar_paths in enumerate(a[paths[0]]):
# #         print(different_similar_paths,total_pos)
# #         print('     ')
# # #         print(work_st_1[index_work_1][1][total_pos+incre][1])
# #         print('     ')
# #     total_pos+=1

#     print('-----')